# Transformers for Language Modelling

As we have familiarized ourselves with the attention mechanism in Transformers, now let's see how these models can be trained as language models.

* In the first part, I prepared a few pretrained language models for you to examine and play with.
* In the second part, we will train a transformer language model from scratch.
  * We will handcraft a simple English language with [Probabilistic Context free Grammar](https://en.wikipedia.org/wiki/Probabilistic_context-free_grammar).
  * Then let the GPT learn this artificial language, and we will examine the emergent phenomena from training.


## Play with Pre-trained transformers language models

`transformers` library from huggingface is the *de facto* standard library for loading and using transformer architectures.

Let's load some common models and see their inner architecture. Specifically, we will look at two classic models, BERT and GPT2. (developped ~ 2018)

As we mentioned in the lecture, they are both transformers but with some prominal differences

* BERT: Bidirectional Encoder Representations from Transformers
  * **Architecture**: All-to-all attention. Transformer Encoder.
  * **Objective**: Masked language modelling
  * **Usage**:
    * Can fill in blanks and perform cloze task
    * Provide representation for many downstream language understanding tasks.
* GPT2: Generative Pre-trained Transformer 2
  * **Architecture**: Causal attention. Transformer Decoder
  * **Objective**: autoregressive language modelling
  * **Usage**:
    * Can continue prompt and answer questions
    * Can be finetuned to follow task specific instructions.


Here are the conceptual picture of BERT and GPT2, let's keep it in mind and try to map them to the modules we see

**BERT**

![BERT (Transformer encoder)](https://iq.opengenus.org/content/images/2020/06/encoder-1.png)

**GPT2**

![](https://miro.medium.com/v2/1*jbcwhhB8PEpJRk781rML_g.png)

### Loading

In [ ]:
import transformers
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config, GPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import pipeline, BertModel, BertConfig, BertTokenizer, BertLMHeadModel

In [ ]:
import torch
import torch.nn as nn
def recursive_print(module, prefix="", depth=0, deepest=3):
    """Simulating print(module) for torch.nn.Modules
        but with depth control. Print to the `deepest` level. `deepest=0` means no print
    """
    if depth == 0:
        print(f"[{type(module).__name__}]")
    if depth >= deepest:
        return
    for name, child in module.named_children():
        if len([*child.named_children()]) == 0:
            print(f"{prefix}({name}): {child}")
        else:
            if isinstance(child, nn.ModuleList):
                print(f"{prefix}({name}): {type(child).__name__} len={len(child)}")
            else:
                print(f"{prefix}({name}): {type(child).__name__}")
        recursive_print(child, prefix + "  ", depth + 1, deepest)

### BERT

In [ ]:
# Accessing the model configuration
BERTtokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
BERTmodel = BertModel.from_pretrained('bert-base-uncased')
BERTconfig = BERTmodel.config

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
BERTconfig

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

### Let BERT do a cloze test

In [ ]:
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

### Inner structure of BERT

In [ ]:
recursive_print(BERTmodel,deepest=1)

In [ ]:
recursive_print(BERTmodel.encoder,deepest=2)

In [ ]:
recursive_print(BERTmodel.encoder.layer[0],deepest=2)

In [ ]:
recursive_print(BERTmodel.encoder.layer[0].attention,deepest=2)

### GPT2

In [ ]:
GPTtokenizer = GPT2Tokenizer.from_pretrained('gpt2')
GPTmodel  = GPT2Model.from_pretrained('gpt2')
GPTconfig = GPT2Config.from_pretrained('gpt2')

In [ ]:
GPTconfig

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.26.1",
  "use_cache": true,
  "vocab_size": 50257
}

### Let GPT say something

In [ ]:
GPTLMmodel = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
input_ids = GPTtokenizer.encode("The meaning of life")
out_ids = GPTLMmodel.generate(torch.tensor(input_ids).long()[None,:],
              max_length=32, do_sample=True, top_k=0, top_p=0.9, temperature=0.7,
              num_return_sequences=5,)
for seq in out_ids:
  print(GPTtokenizer.decode(seq))

### Inner structure of GPTmodel

In [ ]:
recursive_print(GPTmodel,deepest=1)

In [ ]:
recursive_print(GPTmodel.h,deepest=1)

In [ ]:
recursive_print(GPTmodel.h[0],deepest=1)

In [ ]:
recursive_print(GPTmodel.h[0].attn,deepest=2)

In [ ]:
recursive_print(GPTmodel.h[0].mlp,deepest=2)

## Train a transformer to "speak your own language"!

So let's design an artificial language with simple grammar and let the transformer learn to "speak" it. More technically, this means, we hand-craft a probabilistic model of language (sequence) and then let the transformer learn it.

### Probabilistic Generative Grammar

First let's make our *simplified English grammar*.

By building a set of probabilistic generative grammar, or more technically, [Probabilistic context-free grammar, (PCFG)](https://en.wikipedia.org/wiki/Probabilistic_context-free_grammar)

This language has these part of speeches
* Noun
* Intransitive verb
* Transitive verb
* Article
* Adjective
* Conjunction "that"



In [ ]:
#%% list of words and their parts of speech
# nouns
noun_list = ["cat", "dog", "fox", "bird", "horse", "sheep", "cow", "bear", "zebra", "giraffe"]
# intransitive verb
intrans_verb_list = ["ran", "jumped", "swam", "flew", "walked", "slept", "sat", "stood", "danced"]
# transitive verbs that took an object or a clause
trans_verbs_list = ["saw", "heard", "smelled", ]
# adjectives
adj_list = ["big", "small", "tall", "short", "long", "wide", "fat", "thin", "round", "square", "smart", "pretty"]
# adverbs
article_list = ["the", "a"]
# conjunctive that introduces a clause.
conjunction_list = ["that"]

This language also has the following grammar, i.e. a set of substitution rules. For example,

$$S\to NP,VP$$
means a sentence can be substituted by a Noun phrase (NP) + a Verb phrase (VP).
$$VP\to TV, Conj, NP, VP$$
means a verb phrase can be substituted by a transitive verb (TV) + a conjunction (Conj) + a Noun phrase (NP) + a Verb phrase (VP).

The full set of rules are following, feel free to modify it!

In [ ]:
# Rules for mapping part of speech to words
word_map = {
    "N": noun_list,
    "IV": intrans_verb_list,
    "TV": trans_verbs_list,
    "Adj": adj_list,
    "Article": article_list,
    "Conj": conjunction_list,
}
# Grammar for generating sentences
rules = {
    # sentence
    "S": [["NP", "VP"]],
    # noun phrase
    "NP": [["Article", "N"],
           ["Article", "Adj", "N"],
           ["Article", "Adj", "Adj", "N"],
           ["Article", "N", "Conj", "IV"]],
    # verb phrase
    "VP": [["IV"],
           ["TV", "NP"],
           ["TV", "Conj", "NP", "VP"], ],
}

### Sample from probabilistic generative grammar

Now, see what these rules can generate using **probabilistic generative grammar**, i.e. recursively applying the substitution rules to an initial token $S$ to elaborate it.


In [ ]:
import random
def generate_sentences(rules, word_map, max_depth=3, show=False):
    """ A sentence generator with probabilistic generative grammar. """
    initial_token = "S"
    sentence = [initial_token]
    depth = 0
    while True:
        next_sentence = []
        fully_expanded = True
        for token in sentence:
            if token in rules:
                # expand the phrase
                if depth < max_depth:
                    next_sentence.extend(random.choice(rules[token]))
                else:
                    # to limit complexity, we stop adding clauses
                    next_sentence.extend(random.choice(rules[token][:-1]))  # don't expand into the last conjunctive rule

                fully_expanded = False
            else:
                next_sentence.append(token)

        sentence = next_sentence
        depth += 1
        if show:
            print(sentence)
        if fully_expanded:
            break
    # turn tokens into words
    verbal_sentence = []
    for token in sentence:
        word = random.choice(word_map[token])
        verbal_sentence.append(word)
    sent_str = " ".join(verbal_sentence)
    return verbal_sentence, sent_str


word_seq, sentence_str = generate_sentences(rules, word_map, show=True, max_depth=5)
print(sentence_str)

['NP', 'VP']
['Article', 'N', 'Conj', 'IV', 'TV', 'Conj', 'NP', 'VP']
['Article', 'N', 'Conj', 'IV', 'TV', 'Conj', 'Article', 'Adj', 'N', 'IV']
['Article', 'N', 'Conj', 'IV', 'TV', 'Conj', 'Article', 'Adj', 'N', 'IV']
the dog that swam smelled that the round sheep slept


Surely you can generate all kinds of nonsensical sentences which are grammatically correct!
> a small big cow smelled the thin fat zebra

But let's ignore it for a moment and let a transformer learn to "speak" this simply English.

### Tokenization: map words to numbers

For a computational system to understand language, first, we need to break the sentence into words and turn the words into something they know , indices. This is known as tokenization and encoding.

In [ ]:
full_word_set = set(sum([words for words in word_map.values()], []))
dictionary = {word: i for i, word in enumerate(full_word_set)}  # Mapping words to indices
dictionary["[End]"] = len(dictionary)
EOS_ID = dictionary["[End]"]  # end of sentence token
PAD_ID = len(dictionary)  # padding token
dictionary[""] = PAD_ID

inverse_dictionary = {i: word for word, i in dictionary.items()}   # Mapping indices to words

In [ ]:
def tokenize_sentence(sentence):
    """ Tokenize a sentence into a list of words. """
    word_seq = sentence.split(" ")
    return word_seq


def encode_sentence(sentence):
    """ Encode a sentence into a list of indices. """
    word_seq = tokenize_sentence(sentence)
    inds = encode2indices(word_seq)
    return inds


def encode2indices(word_seq):
    """ Encode a list of words into a list of indices. """
    inds = [dictionary[word] for word in word_seq]
    return inds


def decode2words(indices):
    """ Decode a list of indices into a list of words. """
    words = [inverse_dictionary[ind] for ind in indices]
    return words


def decode2sentence(indices):
    """ Decode a list of indices into a sentence. """
    words = decode2words(indices)
    sentence = " ".join(words)
    return sentence

In [ ]:
encode_sentence("a smart fox saw a cat that ran")

[12, 19, 8, 27, 12, 14, 32, 1]

### Build Sequence Dataset: `pad_seq` etc.

Since we don't have a fixed dataset, but a generative model of language, we can simply generate new sentences on the fly and fed it to our transformer. (this is usually not the case)

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

def batch_sampler(batch_size=32, max_len=128):
    batch = []
    for i in range(batch_size):
        word_seq, _ = generate_sentences(rules, word_map)
        word_seq.append("[End]")  # add this marker to say, the sentence has ended.
        inds = encode2indices(word_seq)
        batch.append(torch.tensor(inds, dtype=torch.long))
    # pad the batch to be equal length, same len as the longest senetence.
    padded_batch = pad_sequence(batch, batch_first=True, padding_value=PAD_ID)
    # chuck to the max_len
    padded_batch = padded_batch[:, :max_len]
    return padded_batch

In [ ]:
batch = batch_sampler()
print(batch.shape)
for j in range(batch.shape[0]):
    print(decode2sentence(batch[j].tolist()))

### Train a transformer on this language

In [ ]:
import torch
from torch.optim import AdamW
from transformers import GPT2Model, GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

In [ ]:
miniGPTconfig = GPT2Config(vocab_size=len(dictionary), n_positions=128, n_ctx=128,
                           n_embd=64, n_layer=3, n_head=8,
                           eos_token_id=EOS_ID, pad_token_id=PAD_ID)
miniGPT = GPT2LMHeadModel(miniGPTconfig, )

Finally, let's run our training loops!

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_curve = []
optimizer = AdamW(miniGPT.parameters(), lr=5e-4)
miniGPT.train()
miniGPT.to(device)
batch_size = 512
for epoch in range(3):
    for i in range(50):
        batch = batch_sampler(batch_size=batch_size)
        output = miniGPT(batch.to(device), labels=batch.to(device), )
        loss = output.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"epoch {epoch}, batch {i}, loss {loss.item()}")
        loss_curve.append(loss.item())


miniGPT.eval().to("cpu")

epoch 0, batch 0, loss 3.277460813522339
epoch 0, batch 1, loss 2.996049642562866
epoch 0, batch 2, loss 2.8948705196380615
epoch 0, batch 3, loss 2.891660451889038
epoch 0, batch 4, loss 2.784578323364258
epoch 0, batch 5, loss 2.746859312057495
epoch 0, batch 6, loss 2.7404227256774902
epoch 0, batch 7, loss 2.700308084487915
epoch 0, batch 8, loss 2.6572651863098145
epoch 0, batch 9, loss 2.591931104660034
epoch 0, batch 10, loss 2.575809955596924
epoch 0, batch 11, loss 2.546907901763916
epoch 0, batch 12, loss 2.494894504547119
epoch 0, batch 13, loss 2.4887619018554688
epoch 0, batch 14, loss 2.452965021133423
epoch 0, batch 15, loss 2.3869802951812744
epoch 0, batch 16, loss 2.369704008102417
epoch 0, batch 17, loss 2.3279871940612793
epoch 0, batch 18, loss 2.3047902584075928
epoch 0, batch 19, loss 2.260281801223755
epoch 0, batch 20, loss 2.2382638454437256
epoch 0, batch 21, loss 2.16764235496521
epoch 0, batch 22, loss 2.123156785964966
epoch 0, batch 23, loss 2.11155128479

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(39, 64)
    (wpe): Embedding(128, 64)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.

### Say Something? (or I'm Giving Up On You)

In [ ]:
prompt = "the dog"
prompt_inds = encode_sentence(prompt)
ind_tsr = miniGPT.generate(torch.tensor(prompt_inds).long()[None, :],
                           max_length=32, do_sample=True, top_k=0, top_p=0.9, temperature=0.7,
                           num_return_sequences=5, pad_token_id=PAD_ID)
for ind_seq in ind_tsr:
    print(decode2sentence(ind_seq.tolist()))

the dog that flew heard the short dog [End]           
the dog that slept heard that a round horse saw that the tall bird smelled that a tall fox [End]
the dog that flew saw a pretty bird [End]           
the dog that sat saw a pretty short bear [End]          
the dog that walked jumped [End]              


### Does our transformer "understand" part-of-speech?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

First, let's extract the embeddings of the words and the positions

In [ ]:
token_embedding = miniGPT.transformer.wte.weight.detach()
position_embedding = miniGPT.transformer.wpe.weight.detach()
print(token_embedding.shape)
print(position_embedding.shape)

torch.Size([39, 64])
torch.Size([128, 64])


We'd want to see how these learned embeddings cluster.

In [ ]:
tsne = TSNE(n_components=2, random_state=0, perplexity=15.0)
token_embedding_2d = tsne.fit_transform(token_embedding.numpy())
kmeans = KMeans(n_clusters=6, random_state=0).fit(token_embedding_2d)

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
plt.figure(figsize=[10, 10])
plt.scatter(token_embedding_2d[:, 0], token_embedding_2d[:, 1], c=kmeans.labels_)
# annotate each word on the plot
for i, word in enumerate(inverse_dictionary.values()):
    plt.annotate(word, (token_embedding_2d[i, 0], token_embedding_2d[i, 1]), fontsize=12)
plt.show()

Let's print out which words are in each cluster.

In [ ]:
nCluster = 7
kmeans2 = KMeans(n_clusters=nCluster, random_state=42).fit(token_embedding.numpy())
# print the list of words in each cluster
for icluster in range(nCluster):
    cluster_words = [word for word, ind in dictionary.items() if kmeans2.labels_[ind] == icluster]
    print(f"cluster {icluster}: {cluster_words}")


Overall birds eye view of the word representations (embeddings)

In [ ]:
plt.figure(figsize=[10,5])
plt.imshow(token_embedding)
plt.ylabel("words")
plt.xlabel("embedding dim")
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=[5,10])
plt.imshow(position_embedding)
plt.ylabel("positions")
plt.xlabel("embedding dim")
plt.colorbar()
plt.show()